# Importación de dependencias para el proyecto

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

objc[1146]: Class CaptureDelegate is implemented in both /Users/cristobalnilogarrido/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15850e538) and /Users/cristobalnilogarrido/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1554e0860). One of the two will be used. Which one is undefined.
objc[1146]: Class CVWindow is implemented in both /Users/cristobalnilogarrido/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15850e588) and /Users/cristobalnilogarrido/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x154260a68). One of the two will be used. Which one is undefined.
objc[1146]: Class CVView is implemented in both /Users/cristobalnilogarrido/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15850e5b0) and /Users/cristobalnilogarrido/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x154260a90). One of the two will be used. Which on

# Detección de puntos clave con Mediapipe Holistic

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(imagen,modelo):
    imagen = cv2.cvtColor(imagen,cv2.COLOR_BGR2RGB)
    imagen.flags.writeable=False
    resultados = modelo.process(imagen)
    imagen.flags.writeable=True
    imagen = cv2.cvtColor(imagen,cv2.COLOR_RGB2BGR)
    return imagen,resultados

In [4]:
def draw_styled_landmarks(imagen,resultados):
    mp_drawing.draw_landmarks(imagen,resultados.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness =1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness =1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(imagen,resultados.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness =2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness =2,circle_radius=2)
                             )
    mp_drawing.draw_landmarks(imagen,resultados.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness =2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness =2,circle_radius=2)
                             )
    mp_drawing.draw_landmarks(imagen,resultados.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness =2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,30),thickness =2,circle_radius=2)
                             )

In [30]:
captura = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while captura.isOpened():
    
        # Captura de cámara
        ret,frame=captura.read()

        # Detección
        imagen,resultados=mediapipe_detection(frame,holistic)
        #print(resultados)
        
        draw_styled_landmarks(imagen,resultados)
        cv2.imshow('Traductor', imagen)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
captura.release()
cv2.destroyAllWindows()

# Extracción de puntos clave

In [6]:
pose = []
for resultado in resultados.pose_landmarks.landmark:
    test = np.array([resultado.x, resultado.y, resultado.z, resultado.visibility])
    pose.append(test)

In [7]:
pose = np.array([[resultado.x, resultado.y, resultado.z, resultado.visibility] for resultado in resultados.pose_landmarks.landmark]).flatten() if resultados.pose_landmarks else np.zeros(132)
face = np.array([[resultado.x, resultado.y, resultado.z] for resultado in resultados.face_landmarks.landmark]).flatten() if resultados.face_landmarks else np.zeros(1404)
lh = np.array([[resultado.x, resultado.y, resultado.z] for resultado in resultados.left_hand_landmarks.landmark]).flatten() if resultados.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[resultado.x, resultado.y, resultado.z] for resultado in resultados.right_hand_landmarks.landmark]).flatten() if resultados.right_hand_landmarks else np.zeros(21*3)


In [8]:
#face = np.array([[resultado.x, resultado.y, resultado.z] for resultado in resultados.face_landmarks.landmark]).flatten() 
#if resultados.face_landmarks 
#else np.zeros(1404)

In [9]:
def extract_keypoints(resultados):
    pose = np.array([[resultado.x, resultado.y, resultado.z, resultado.visibility] for resultado in resultados.pose_landmarks.landmark]).flatten() if resultados.pose_landmarks else np.zeros(132)
    face = np.array([[resultado.x, resultado.y, resultado.z] for resultado in resultados.face_landmarks.landmark]).flatten() if resultados.face_landmarks else np.zeros(1404)
    lh = np.array([[resultado.x, resultado.y, resultado.z] for resultado in resultados.left_hand_landmarks.landmark]).flatten() if resultados.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[resultado.x, resultado.y, resultado.z] for resultado in resultados.right_hand_landmarks.landmark]).flatten() if resultados.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [10]:
result_test = extract_keypoints(resultados)

In [11]:
np.save('0', result_test)

In [12]:
np.load('0.npy')

array([ 0.55645436,  0.52572334, -1.72443449, ...,  0.        ,
        0.        ,  0.        ])

# Directorios para el almacenamiento de los datos

In [14]:

DATA_PATH = os.path.join('Data') 


actions = np.array(['Hola', '¿Cómo estás?', '¿Cuál?'])

no_sequences = 30

sequence_length = 30

In [14]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Creación de videos con el DataSet para entrenamiento y testing

In [35]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'EMPEZANDO LA COLECCIÓN', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Recolectando cuadros para {} Video Número {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Recolectando cuadros para {} Video Número {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [16]:
cap.release()
cv2.destroyAllWindows()

# Procesamiento de datos para el modelo.

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
label_map

{'Hola': 0, '¿Cómo estás?': 1, '¿Cuál?': 2}

In [17]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [18]:
np.array(sequences).shape

(90, 30, 1662)

In [19]:
np.array(labels).shape

(90,)

In [20]:
X = np.array(sequences)

In [21]:
y = to_categorical(labels).astype(int)

NameError: name 'to_categorical' is not defined

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Creación y entrenamiento de la LSTM

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [13]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [29]:
res = [.7, 0.2, 0.1]

In [12]:
actions[np.argmax(res)]

NameError: name 'actions' is not defined

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [32]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


2022-10-13 13:26:13.773915: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 2s 30ms/step - loss: 1.5127 - categorical_accuracy: 0.4000
Epoch 2/2000
3/3 [==============================] - 0s 37ms/step - loss: 11.2324 - categorical_accuracy: 0.4000
Epoch 3/2000
3/3 [==============================] - 0s 37ms/step - loss: 12.1142 - categorical_accuracy: 0.3647
Epoch 4/2000
3/3 [==============================] - 0s 34ms/step - loss: 24.6608 - categorical_accuracy: 0.2824
Epoch 5/2000
3/3 [==============================] - 0s 34ms/step - loss: 9.2079 - categorical_accuracy: 0.3059
Epoch 6/2000
3/3 [==============================] - 0s 35ms/step - loss: 5.2444 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 0s 33ms/step - loss: 3.4136 - categorical_accuracy: 0.3176
Epoch 8/2000
3/3 [==============================] - 0s 35ms/step - loss: 2.9915 - categorical_accuracy: 0.3647
Epoch 9/2000
3/3 [==============================] - 0s 35ms/step - loss: 3.2940 - categorical_accuracy: 0.3647
Epoch 10/20

3/3 [==============================] - 0s 51ms/step - loss: 1.0814 - categorical_accuracy: 0.3882
Epoch 147/2000
3/3 [==============================] - 0s 49ms/step - loss: 1.1555 - categorical_accuracy: 0.3412
Epoch 148/2000
3/3 [==============================] - 0s 47ms/step - loss: 1.1041 - categorical_accuracy: 0.4706
Epoch 149/2000
3/3 [==============================] - 0s 45ms/step - loss: 1.0396 - categorical_accuracy: 0.5294
Epoch 150/2000
3/3 [==============================] - 0s 47ms/step - loss: 1.0836 - categorical_accuracy: 0.3882
Epoch 151/2000
3/3 [==============================] - 0s 46ms/step - loss: 1.0802 - categorical_accuracy: 0.4706
Epoch 152/2000
3/3 [==============================] - 0s 42ms/step - loss: 1.0334 - categorical_accuracy: 0.4118
Epoch 153/2000
3/3 [==============================] - 0s 43ms/step - loss: 1.0029 - categorical_accuracy: 0.4706
Epoch 154/2000
3/3 [==============================] - 0s 46ms/step - loss: 1.0318 - categorical_accuracy: 0.423

3/3 [==============================] - 0s 47ms/step - loss: 0.1024 - categorical_accuracy: 0.9765
Epoch 291/2000
3/3 [==============================] - 0s 52ms/step - loss: 0.0796 - categorical_accuracy: 0.9882
Epoch 292/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.0940 - categorical_accuracy: 0.9765
Epoch 293/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.0907 - categorical_accuracy: 0.9647
Epoch 294/2000
3/3 [==============================] - 0s 51ms/step - loss: 0.0669 - categorical_accuracy: 0.9765
Epoch 295/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.0583 - categorical_accuracy: 0.9765
Epoch 296/2000
3/3 [==============================] - 0s 55ms/step - loss: 0.0656 - categorical_accuracy: 0.9765
Epoch 297/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0520 - categorical_accuracy: 0.9765
Epoch 298/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.0503 - categorical_accuracy: 0.976

...

3/3 [==============================] - 0s 46ms/step - loss: 0.0066 - categorical_accuracy: 1.0000
Epoch 1515/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.0059 - categorical_accuracy: 1.0000
Epoch 1516/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.0061 - categorical_accuracy: 1.0000
Epoch 1517/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.0065 - categorical_accuracy: 1.0000
Epoch 1518/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.0045 - categorical_accuracy: 1.0000
Epoch 1519/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.0058 - categorical_accuracy: 1.0000
Epoch 1520/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0044 - categorical_accuracy: 1.0000
Epoch 1521/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.0049 - categorical_accuracy: 1.0000
Epoch 1522/2000
3/3 [==============================] - 0s 49ms/step - loss: 0.0046 - categorical_accurac

3/3 [==============================] - 0s 51ms/step - loss: 0.0032 - categorical_accuracy: 1.0000
Epoch 1587/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0038 - categorical_accuracy: 1.0000
Epoch 1588/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.0033 - categorical_accuracy: 1.0000
Epoch 1589/2000
3/3 [==============================] - 0s 49ms/step - loss: 0.0044 - categorical_accuracy: 1.0000
Epoch 1590/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0032 - categorical_accuracy: 1.0000
Epoch 1591/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 1592/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0036 - categorical_accuracy: 1.0000
Epoch 1593/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.0031 - categorical_accuracy: 1.0000
Epoch 1594/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0031 - categorical_accurac

3/3 [==============================] - 0s 42ms/step - loss: 0.0022 - categorical_accuracy: 1.0000
Epoch 1659/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1660/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.0022 - categorical_accuracy: 1.0000
Epoch 1661/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1662/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.0022 - categorical_accuracy: 1.0000
Epoch 1663/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1664/2000
3/3 [==============================] - 0s 49ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1665/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1666/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.0021 - categorical_accurac

Epoch 1730/2000
3/3 [==============================] - 0s 51ms/step - loss: 9.6307e-04 - categorical_accuracy: 1.0000
Epoch 1731/2000
3/3 [==============================] - 0s 53ms/step - loss: 9.4112e-04 - categorical_accuracy: 1.0000
Epoch 1732/2000
3/3 [==============================] - 0s 50ms/step - loss: 9.5663e-04 - categorical_accuracy: 1.0000
Epoch 1733/2000
3/3 [==============================] - 0s 48ms/step - loss: 9.7303e-04 - categorical_accuracy: 1.0000
Epoch 1734/2000
3/3 [==============================] - 0s 55ms/step - loss: 9.3636e-04 - categorical_accuracy: 1.0000
Epoch 1735/2000
3/3 [==============================] - 0s 59ms/step - loss: 9.6554e-04 - categorical_accuracy: 1.0000
Epoch 1736/2000
3/3 [==============================] - 0s 65ms/step - loss: 9.9627e-04 - categorical_accuracy: 1.0000
Epoch 1737/2000
3/3 [==============================] - 0s 58ms/step - loss: 9.2642e-04 - categorical_accuracy: 1.0000
Epoch 1738/2000
3/3 [==============================] - 0

3/3 [==============================] - 0s 51ms/step - loss: 6.0626e-04 - categorical_accuracy: 1.0000
Epoch 1869/2000
3/3 [==============================] - 0s 51ms/step - loss: 6.0998e-04 - categorical_accuracy: 1.0000
Epoch 1870/2000
3/3 [==============================] - 0s 47ms/step - loss: 6.0760e-04 - categorical_accuracy: 1.0000
Epoch 1871/2000
3/3 [==============================] - 0s 49ms/step - loss: 5.9668e-04 - categorical_accuracy: 1.0000
Epoch 1872/2000
3/3 [==============================] - 0s 48ms/step - loss: 6.1790e-04 - categorical_accuracy: 1.0000
Epoch 1873/2000
3/3 [==============================] - 0s 50ms/step - loss: 6.0605e-04 - categorical_accuracy: 1.0000
Epoch 1874/2000
3/3 [==============================] - 0s 52ms/step - loss: 6.0102e-04 - categorical_accuracy: 1.0000
Epoch 1875/2000
3/3 [==============================] - 0s 46ms/step - loss: 5.9454e-04 - categorical_accuracy: 1.0000
Epoch 1876/2000
3/3 [==============================] - 0s 50ms/step - lo

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# Predicciones

In [34]:
res = model.predict(X_test)

1/1 [==============================] - 0s 205ms/step


In [35]:
actions[np.argmax(res[4])]

'¿Cuál?'

In [36]:
actions[np.argmax(y_test[4])]

'¿Cuál?'

# Almacenar pesos

In [37]:
model.save('action.h5')

In [31]:
model.load_weights('action.h5')

# Evaluación del modelo

# Pruebas en tiempo real

In [32]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [33]:
plt.figure(figsize=(18,18))

<Figure size 1800x1800 with 0 Axes>

<Figure size 1800x1800 with 0 Axes>

In [22]:
sequence.append('def')

NameError: name 'sequence' is not defined

In [42]:
sequence.reverse()

AttributeError: 'int' object has no attribute 'reverse'

In [43]:
sequence[-30:]

TypeError: 'int' object is not subscriptable

In [34]:

sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

     
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        

        draw_styled_landmarks(image, results)
        

        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
    
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

           
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
     
        cv2.imshow('OpenCV Feed', image)

      
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 11ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [====================

1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [=========================

1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [=========================

1/1 [==============================] - 0s 10ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.

1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<cla

1/1 [==============================] - 0s 10ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==========================

1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution

1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 10ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
Hola
<class 'mediapipe.python.solution_b

1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cuál?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 9ms/step
¿Cómo estás?
<cla

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))